In [1]:
import os

import tensorflow as tf
import keras 

import numpy as np

from PIL import Image

import winsound

from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import concatenate

from tensorflow.keras import Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.keras.backend as K

from tensorflow.keras.backend import set_session, clear_session, get_session

Using TensorFlow backend.


In [2]:
# global variables

standard_epochs = 100  # epochs for unaugmented data
augmented_epochs = 100 # epochs for augmented data
resize_shape=(256, 256)
batch_size = 10
steps_per_epoch = int(1500 / batch_size) + 1 

weights_file = "highest.hdf5"

predictions_dir = "predictions/"
image_subset_dir = "subset_files/"

training_images_dir = "training_validation_Hair_Skin_Masks/training_hair_skin/images/images/"
training_images_flow_dir = "training_validation_Hair_Skin_Masks/training_hair_skin/images/"
testing_images_dir = "testing_images/testing_images/"
validation_images_dir = "training_validation_Hair_Skin_Masks/validation_hair_skin/images/"

training_masks_dir = "training_validation_Hair_Skin_Masks/training_hair_skin/masks/masks/"
training_masks_flow_dir = "training_validation_Hair_Skin_Masks/training_hair_skin/masks/"
validation_masks_dir = "training_validation_Hair_Skin_Masks/validation_hair_skin/masks/"

In [3]:
# calculate the weighted categorical crossentropy
# for evaluation with a keras model
#

def weighted_cce(y_true, y_pred, weights=np.array([4,1,1])):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    # scale predictions so that the class probas of each sample sum to 1
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
    # clip to prevent NaN's and Inf's
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # calc
    loss = y_true * K.log(y_pred) * weights
    loss = -K.sum(loss, -1)
    
    weights = K.variable(weights)
    
    return loss

In [4]:
# calculate the dice coefficient of a 
# predicted and true image tensors
#

def dice_coef(y_true, y_pred, smooth=1):
    
    # calculate the magnitude of the intersection between
    # the predicted and true tensors
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    
    # calculate the magnitude of the union of the 
    # predicted and true tensors
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

In [5]:
# calculate the dice loss for the 
# true and predicted image tensors
#

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [6]:
# custom implementation of the FCN8 segmentation 
# architecture
#

def fcn8(n_classes, input_size = (256,256,3)):
    
    # input layer
    inputs = Input(input_size)
    
    # layer 1 (256 x 256)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # layer 2 (128 x 128)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # layer 3 (64 x 64)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)  
    predict_64x64 = Conv2D(n_classes, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)     
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    # layer 4 (32 x 32)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    predict_32x32 = Conv2D(n_classes, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # layer 5 (16 x 16)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    conv5 = Conv2D(n_classes, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    
    # layer 7 (upsampling to 32 x 32)
    up7 = Conv2D(n_classes, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
    
    # add layer 4 prediction to layer 7
    merge7 = concatenate([predict_32x32,up7], axis = 3)
    
    # layer 8 (upsampling to 64 x 64)
    up8 = Conv2D(n_classes, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(merge7))
    
    # add layer 3 prediction to layer 8
    merge8 = concatenate([predict_64x64, up8], axis = 3)
    
    # layer 9 (upsampling to 256 x 256)
    up9 = Conv2D(n_classes, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(merge8))
    up9 = Conv2D(n_classes, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(up9))
    
    # output layer
    outputs = Conv2D(n_classes, 1, activation = 'softmax')(up9)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [8]:
# custom implementation of the U-Net segmentation
# architecture
#

def unet(n_classes, input_size = (256,256,3)):
    
    # input layer
    inputs = Input(input_size)
    
    # layer 1
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # layer 2
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # layer 3
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    # layer 4 
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # layer 5
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)

    # layer 6 (upsampling)
    up6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
    
    # take information from layer 4
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    
    # layer 7 (upsampling)
    up7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    
    # take information from layer 3
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    # layer 8 (upsampling)
    up8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    
    # take information from layer 2
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    # layer 9 (upsampling)
    up9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    
    # take information from layer 1
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    
    # output layer
    outputs = Conv2D(n_classes, 1, activation='softmax')(conv9)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [7]:
# delete all imagese in the predictions 
# directory
# 
# @predictions_dir: the directory path to be 
#                   cleared
#

def clear_old_predictions(predictions_dir):
    files = os.listdir(predictions_dir)
    for file in files:
        file_path = os.path.join(predictions_dir, file)
        os.remove(file_path)

In [9]:
# return images from the directory pointed to from dir_path
# as an Numpy array of shape (# of images, image width, image height, channels)
#
# @image_files: an array of strings containing the filepaths to the images
#               to be read from the directory. Used in conjunction with 
#               the files created by get_appropriate_images.ipynb
# @is_mask: If true, images will be converted to binary images and channels 
#           will be set to 1
# 

def get_np_array_images_from_directory(dir_path, image_files=None):
    
    # container for image arrays
    np_arrays = []
    
    # if image_files wasn't specified read all images from directory
    if image_files is None:
        image_files = os.listdir(dir_path)
        image_files = sorted(image_files, key=lambda s:int(s.split('_')[2].split('.')[0]))
    
    # construct the appropriate absolute paths for each image
    image_paths = [os.path.join(dir_path, filename) for filename in image_files]
    
    # read, resize, and convert each image to a Numpy array
    for path in image_paths:
        
        # open image
        img = Image.open(path)
        
        # resize image
        img = img.resize((256, 256))
        
#         # convert to binary image if is_mask is True
#         if is_mask:
#             img = img.convert("1")
        
        # convert image to Numpy array 
        array = np.array(img)
        
#         # ensure mask images have a single color channel dimension
#         if is_mask:
#             array = array.reshape(256, 256, 1)
        
        # add the image to the image arrays container
        np_arrays.append(array)
        
    return np.array(np_arrays)

In [10]:
# convert a Numpy array containing the information 
# for a mask image to a pillow image
# 
# @np_array: the array containing the binary mask
#            information
#
# @return: a Pillow Image
#

def get_mask_from_binary_array(np_array):
    
    # extract dimensions of image
    height = np_array.shape[0]
    width = np_array.shape[1]
    
    # cut off the channels dimension
    # which should be 1
    np_array = np_array.reshape(height, width)
    
    # convert to 255 black and white representation
    np_array *= 255
    
    return Image.fromarray(np_array)

In [11]:
# set gpu configuration for this system
#

config = tf.compat.v1.ConfigProto( device_count={'GPU': 1, 'CPU':8})
sess = tf.compat.v1.Session(config=config)
keras.backend.set_session(sess)

In [12]:
# read in the validation and testing images and masks
#

training_image_arrays = get_np_array_images_from_directory(training_images_dir)
training_mask_arrays = get_np_array_images_from_directory(training_masks_dir)
training_image_arrays = training_image_arrays.astype("float32") / 255.0
training_mask_arrays = training_mask_arrays / 255
training_mask_arrays = training_mask_arrays.astype("int")

validation_image_arrays = get_np_array_images_from_directory(validation_images_dir)
validation_mask_arrays = get_np_array_images_from_directory(validation_masks_dir)
validation_image_arrays = validation_image_arrays.astype("float32") / 255.0
validation_mask_arrays = validation_mask_arrays / 255
validation_mask_arrays = validation_mask_arrays.astype("int")

testing_image_arrays = get_np_array_images_from_directory(testing_images_dir)
testing_image_arrays = testing_image_arrays.astype("float32") / 255.0

In [14]:
# map of the custom metrics
# 
# required information for models
#

dependencies = {
    "dice_coef": dice_coef,
    "dice_loss": dice_loss,
    "weighted_cce": weighted_cce
}

In [15]:
# initialize the model
#

# model = fcn8(3)
model = unet(3)

# optimizer = Adam(lr=1e-4) # unet with categorical crossentropy
optimizer = Adam(lr=1e-4) # unet with weighted categorical crossentropy [2.0,1,1]
# optimizer = Adam(lr=5e-5) # unet with dice loss
# optimizer = Adam(lr=1e-3) # fcn8 with categorical crossentropy
# optimizer = Adam(lr=1e-4) # fcn8 with dice loss
# optimizer = Adam(lr=1E-4) # fcn8 with weighted categorical crossentropy [1.5,1,1]
model.compile(optimizer=optimizer, loss=weighted_cce, metrics = [dice_coef])

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
_____________

In [16]:
# the augmentation generators for training
#

datagen_args = dict(
    rotation_range=45,
    rescale=1.0/255.0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2
)

# augmentation for images and masks must be
# identical
#

train_image_datagen = ImageDataGenerator(**datagen_args, dtype="float32")
train_mask_datagen = ImageDataGenerator(**datagen_args, dtype="int")

seed = 1337
train_image_datagen.fit(training_image_arrays, augment=True, seed=seed)
train_mask_datagen.fit(training_mask_arrays, augment=True, seed=seed)

In [17]:
# create one generator for the training images
# and another for the training masks
#

generator_args = dict(
    class_mode=None,
    target_size=resize_shape,
    batch_size=batch_size,
    seed=seed
)

train_image_generator = train_image_datagen.flow_from_directory(
    training_images_flow_dir,
    **generator_args
)
train_mask_generator = train_mask_datagen.flow_from_directory(
    training_masks_flow_dir,
    **generator_args
)

train_generator = (pair for pair in zip(train_image_generator, train_mask_generator))

Found 1500 images belonging to 1 classes.
Found 1500 images belonging to 1 classes.


In [18]:
# train the model on unaugmented data first and then
# on augmented data
# 

# create the checkpoints for the model
#
checkpoint = ModelCheckpoint(
    weights_file, 
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    save_best_only=True
)
earlystop = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=10,
    restore_best_weights=True
)

callbacks = [checkpoint, earlystop]

# train on unaltered images
#
model.fit(
    x=training_image_arrays, y=training_mask_arrays,
    validation_data=(validation_image_arrays, validation_mask_arrays),
    epochs=standard_epochs,
    batch_size=batch_size,
    callbacks=callbacks
)

# train on augmented data
#
model.fit_generator(
    train_generator, 
    epochs=augmented_epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=(validation_image_arrays, validation_mask_arrays),
    callbacks=callbacks
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1500 samples, validate on 500 samples
Epoch 1/100
1490/1500 [============================>.] - ETA: 0s - loss: 0.8030 - dice_coef: 0.6702
Epoch 00001: val_loss improved from inf to 0.59233, saving model to highest.hdf5
1500/1500 [==============================] - 147s 98ms/sample - loss: 0.8014 - dice_coef: 0.6708 - val_loss: 0.5923 - val_dice_coef: 0.8045
Epoch 2/100
1490/1500 [============================>.] - ETA: 0s - loss: 0.5602 - dice_coef: 0.7841
Epoch 00002: val_loss improved from 0.59233 to 0.54224, saving model to highest.hdf5
1500/1500 [==============================] - 142s 94ms/sample - loss: 0.5606 - dice_coef: 0.7837 - val_loss: 0.5422 - val_dice_coef: 0.7702
Epoch 3/100
1490/1500 [============================>.] - ETA: 0s - loss: 0.5023 - dice_coef: 0.8081
Epoch 00003: val_loss improved from 0.54224 to 0.50978, saving model to highest.hdf5
1500/1500 [=================

Epoch 2/100
500/151 [===================================================================================================] - 14s 28ms/sample - loss: 0.2795 - dice_coef: 0.8858

Epoch 00002: val_loss did not improve from 0.22764
151/151 [==============================] - 139s 919ms/step - loss: 0.3230 - dice_coef: 0.8779 - val_loss: 0.2774 - val_dice_coef: 0.8858
Epoch 3/100
500/151 [===================================================================================================] - 14s 28ms/sample - loss: 0.2506 - dice_coef: 0.8865

Epoch 00003: val_loss did not improve from 0.22764
151/151 [==============================] - 139s 919ms/step - loss: 0.2907 - dice_coef: 0.8912 - val_loss: 0.2524 - val_dice_coef: 0.8865
Epoch 4/100
500/151 [===================================================================================================] - 14s 28ms/sample - loss: 0.2614 - dice_coef: 0.9074

Epoch 00004: val_loss did not improve from 0.22764
151/151 [==============================] - 13

In [19]:
# load the model with the lowest validation loss
#

loaded_model = load_model(weights_file, custom_objects=dependencies)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [20]:
# make predictions
#
predictions = loaded_model.predict(testing_image_arrays, batch_size=1)

# only classify the areas containing hair
# all other areas should be black
#
classes = (predictions.argmax(axis=-1) == 0).astype("int")

In [21]:
# check for correct shapes
#
print(classes.shape)
print(predictions.shape)

(927, 256, 256)
(927, 256, 256, 3)


In [22]:
# visually inspect the results of the 
# prediction for variation 
# 
# constant values indicate a problem with
# the training
#
for row in predictions[0]:
    print(row)

[[7.5998940e-02 5.4616630e-03 9.1853940e-01]
 [3.5013903e-02 7.0207193e-04 9.6428406e-01]
 [2.5561079e-02 2.5032891e-04 9.7418857e-01]
 [1.3265696e-02 7.3217452e-05 9.8666102e-01]
 [7.8465603e-03 3.7016081e-05 9.9211639e-01]
 [6.1750086e-03 2.6597007e-05 9.9379843e-01]
 [5.2906550e-03 2.4808247e-05 9.9468452e-01]
 [4.8550186e-03 2.3276089e-05 9.9512166e-01]
 [4.5407414e-03 2.1025960e-05 9.9543828e-01]
 [4.4150888e-03 1.9880634e-05 9.9556500e-01]
 [4.2179874e-03 1.8746163e-05 9.9576318e-01]
 [4.1957451e-03 1.8699211e-05 9.9578553e-01]
 [4.1542696e-03 1.9248260e-05 9.9582642e-01]
 [4.0849638e-03 1.9627067e-05 9.9589539e-01]
 [3.9911144e-03 2.0792457e-05 9.9598807e-01]
 [3.9649899e-03 2.1924659e-05 9.9601305e-01]
 [3.9949417e-03 2.2799986e-05 9.9598229e-01]
 [4.0000821e-03 2.3839086e-05 9.9597615e-01]
 [4.0254439e-03 2.5190717e-05 9.9594939e-01]
 [4.0228725e-03 2.5799462e-05 9.9595129e-01]
 [4.0086601e-03 2.6006890e-05 9.9596536e-01]
 [3.9761742e-03 2.5459052e-05 9.9599838e-01]
 [3.933396

[[1.06467940e-02 1.82116427e-03 9.87532079e-01]
 [5.98306488e-03 1.42903952e-03 9.92587864e-01]
 [3.74316005e-03 1.10048067e-03 9.95156407e-01]
 [2.90643377e-03 6.54388394e-04 9.96439159e-01]
 [2.30028131e-03 5.10614249e-04 9.97189105e-01]
 [2.57192459e-03 3.26270820e-04 9.97101843e-01]
 [2.27995845e-03 2.12581886e-04 9.97507453e-01]
 [2.09809910e-03 1.08434593e-04 9.97793436e-01]
 [2.02446920e-03 6.40851140e-05 9.97911394e-01]
 [1.88834406e-03 4.40625699e-05 9.98067558e-01]
 [1.75174593e-03 3.43438187e-05 9.98213887e-01]
 [1.77738094e-03 3.01037126e-05 9.98192489e-01]
 [1.80409721e-03 3.03245070e-05 9.98165548e-01]
 [1.76294206e-03 3.27253765e-05 9.98204350e-01]
 [1.73553207e-03 3.59843325e-05 9.98228490e-01]
 [1.67302042e-03 3.83560509e-05 9.98288572e-01]
 [1.61349098e-03 3.69049558e-05 9.98349667e-01]
 [1.57793623e-03 3.50434675e-05 9.98386979e-01]
 [1.55206898e-03 3.26766785e-05 9.98415232e-01]
 [1.52416690e-03 3.20392792e-05 9.98443782e-01]
 [1.52660511e-03 3.30086332e-05 9.984404

[[1.06558520e-02 2.35748803e-03 9.86986697e-01]
 [5.56191616e-03 2.02191714e-03 9.92416143e-01]
 [3.35511914e-03 1.64775830e-03 9.94997144e-01]
 [2.49343575e-03 1.11335202e-03 9.96393263e-01]
 [1.83479395e-03 9.53605340e-04 9.97211635e-01]
 [1.98457506e-03 7.00549223e-04 9.97314870e-01]
 [1.66704634e-03 4.99332324e-04 9.97833669e-01]
 [1.48399279e-03 2.69579468e-04 9.98246431e-01]
 [1.40050787e-03 1.59778137e-04 9.98439729e-01]
 [1.27503625e-03 1.09680703e-04 9.98615265e-01]
 [1.18062808e-03 8.92749158e-05 9.98730123e-01]
 [1.17067667e-03 8.00550624e-05 9.98749256e-01]
 [1.16166007e-03 8.28248303e-05 9.98755455e-01]
 [1.10964256e-03 9.39090751e-05 9.98796463e-01]
 [1.04823511e-03 1.07551008e-04 9.98844147e-01]
 [9.70638881e-04 1.18095813e-04 9.98911262e-01]
 [9.02790460e-04 1.15980467e-04 9.98981178e-01]
 [8.55786144e-04 1.11317800e-04 9.99032855e-01]
 [8.21216905e-04 1.03695129e-04 9.99075055e-01]
 [7.91943807e-04 1.01981663e-04 9.99105990e-01]
 [7.80058734e-04 1.03673687e-04 9.991162

[[1.08940741e-02 3.23800812e-03 9.85867918e-01]
 [4.63073002e-03 3.20734666e-03 9.92161930e-01]
 [2.23751180e-03 2.27238634e-03 9.95490134e-01]
 [1.68591877e-03 1.88686664e-03 9.96427238e-01]
 [1.21964328e-03 1.44208909e-03 9.97338235e-01]
 [1.26728474e-03 1.16284157e-03 9.97569859e-01]
 [1.18813000e-03 8.85941379e-04 9.97925878e-01]
 [1.09562220e-03 6.14086573e-04 9.98290241e-01]
 [1.01327070e-03 4.34873538e-04 9.98551786e-01]
 [9.29385482e-04 3.69509711e-04 9.98701096e-01]
 [8.34125734e-04 3.29922943e-04 9.98835981e-01]
 [8.14203057e-04 3.04748246e-04 9.98881042e-01]
 [8.18689936e-04 2.90599070e-04 9.98890698e-01]
 [8.25725088e-04 2.82098801e-04 9.98892248e-01]
 [7.66721962e-04 2.69062875e-04 9.98964190e-01]
 [6.95511932e-04 2.51474412e-04 9.99053061e-01]
 [6.14172604e-04 2.32644365e-04 9.99153137e-01]
 [5.48039097e-04 2.11791630e-04 9.99240160e-01]
 [4.90409380e-04 2.05396733e-04 9.99304175e-01]
 [4.46914259e-04 1.95069297e-04 9.99357998e-01]
 [4.00643738e-04 1.81549214e-04 9.994177

[[8.20614770e-03 9.59782861e-04 9.90834057e-01]
 [4.68438072e-03 9.64457286e-04 9.94351149e-01]
 [2.80767516e-03 6.27030793e-04 9.96565282e-01]
 [2.41229031e-03 5.03488001e-04 9.97084200e-01]
 [2.18129065e-03 4.38865070e-04 9.97379839e-01]
 [2.26723216e-03 3.64222884e-04 9.97368574e-01]
 [1.79127627e-03 3.27088550e-04 9.97881591e-01]
 [1.41793024e-03 2.55769701e-04 9.98326242e-01]
 [1.16195751e-03 1.92067499e-04 9.98645961e-01]
 [1.12375291e-03 1.50398482e-04 9.98725832e-01]
 [1.04824267e-03 1.10476365e-04 9.98841345e-01]
 [9.84288054e-04 7.93939253e-05 9.98936355e-01]
 [9.02780273e-04 6.53597090e-05 9.99031901e-01]
 [8.15683859e-04 5.61083034e-05 9.99128163e-01]
 [7.27521430e-04 5.11620710e-05 9.99221325e-01]
 [6.52998162e-04 4.44133984e-05 9.99302626e-01]
 [5.61431400e-04 3.67594439e-05 9.99401808e-01]
 [5.10114653e-04 3.11477124e-05 9.99458730e-01]
 [4.52602108e-04 2.69347620e-05 9.99520421e-01]
 [4.12709283e-04 2.40937406e-05 9.99563158e-01]
 [3.61373997e-04 2.37053519e-05 9.996148

...

[[1.52197868e-01 3.36698699e-03 8.44435155e-01]
 [1.39280617e-01 1.29077409e-03 8.59428644e-01]
 [1.15907125e-01 5.44980285e-04 8.83547902e-01]
 [1.46923974e-01 6.69643865e-04 8.52406383e-01]
 [1.68685928e-01 7.18468567e-04 8.30595613e-01]
 [1.72627985e-01 8.81628250e-04 8.26490402e-01]
 [1.68954611e-01 1.12520473e-03 8.29920173e-01]
 [1.58607841e-01 1.41376164e-03 8.39978397e-01]
 [1.52979538e-01 1.88685581e-03 8.45133543e-01]
 [1.46995768e-01 2.18630233e-03 8.50817919e-01]
 [1.39779031e-01 2.41460674e-03 8.57806325e-01]
 [1.36960104e-01 2.58037914e-03 8.60459566e-01]
 [1.20296970e-01 2.75580492e-03 8.76947224e-01]
 [1.06973894e-01 3.34268552e-03 8.89683425e-01]
 [9.25000831e-02 4.43365891e-03 9.03066278e-01]
 [8.14649090e-02 6.09875191e-03 9.12436366e-01]
 [7.20745474e-02 7.72790611e-03 9.20197546e-01]
 [6.07787929e-02 8.43486004e-03 9.30786371e-01]
 [5.14265224e-02 7.39369867e-03 9.41179752e-01]
 [4.36649993e-02 6.29226863e-03 9.50042665e-01]
 [3.66380848e-02 5.64480014e-03 9.577171

[[2.65852422e-01 1.95920877e-02 7.14555442e-01]
 [2.48001337e-01 9.00425296e-03 7.42994368e-01]
 [2.36091614e-01 5.76011278e-03 7.58148253e-01]
 [2.51762658e-01 5.27537195e-03 7.42961943e-01]
 [2.57729918e-01 4.93242824e-03 7.37337649e-01]
 [2.59648651e-01 5.60689624e-03 7.34744430e-01]
 [2.44496211e-01 6.37911633e-03 7.49124646e-01]
 [2.38332465e-01 7.10545527e-03 7.54562020e-01]
 [2.36260056e-01 8.24875012e-03 7.55491138e-01]
 [2.20160708e-01 8.96391273e-03 7.70875394e-01]
 [2.06449926e-01 1.05815968e-02 7.82968462e-01]
 [1.98979378e-01 1.28794750e-02 7.88141131e-01]
 [1.92952648e-01 1.43601988e-02 7.92687178e-01]
 [1.94194779e-01 1.58397406e-02 7.89965510e-01]
 [1.89957842e-01 1.64624173e-02 7.93579757e-01]
 [1.64611489e-01 1.70515459e-02 8.18336904e-01]
 [1.44443214e-01 1.98131595e-02 8.35743666e-01]
 [9.81369019e-02 2.24711448e-02 8.79391968e-01]
 [5.71208000e-02 2.13103816e-02 9.21568811e-01]
 [2.35794969e-02 1.22398240e-02 9.64180648e-01]
 [1.09767187e-02 6.73093880e-03 9.822923

 [1.25529559e-03 7.08890320e-06 9.98737633e-01]]
[[6.37801230e-01 3.04646771e-02 3.31734061e-01]
 [6.39910340e-01 1.16436351e-02 3.48446012e-01]
 [6.24582887e-01 8.84323660e-03 3.66573870e-01]
 [6.25878692e-01 8.98155384e-03 3.65139753e-01]
 [6.04807794e-01 9.58352722e-03 3.85608643e-01]
 [5.87441385e-01 1.19641041e-02 4.00594562e-01]
 [5.71114600e-01 1.49384569e-02 4.13946927e-01]
 [5.57965159e-01 2.09987313e-02 4.21036124e-01]
 [5.61300397e-01 2.50839535e-02 4.13615674e-01]
 [5.56513429e-01 2.93760765e-02 4.14110512e-01]
 [5.57499886e-01 2.89512891e-02 4.13548827e-01]
 [5.58931112e-01 3.59908976e-02 4.05077994e-01]
 [5.39448202e-01 5.40067442e-02 4.06545103e-01]
 [4.16464865e-01 9.86912027e-02 4.84843940e-01]
 [2.55495965e-01 1.31280839e-01 6.13223255e-01]
 [1.17205560e-01 1.17891282e-01 7.64903128e-01]
 [6.46597594e-02 8.78432617e-02 8.47496986e-01]
 [4.45913374e-02 7.21499920e-02 8.83258700e-01]
 [3.42195854e-02 7.76138008e-02 8.88166666e-01]
 [2.93111037e-02 8.83487910e-02 8.82340

[[6.29663825e-01 4.03690264e-02 3.29967141e-01]
 [6.30957425e-01 2.14626435e-02 3.47579926e-01]
 [6.28597438e-01 1.89974438e-02 3.52405161e-01]
 [6.32203400e-01 2.12281160e-02 3.46568465e-01]
 [6.05144620e-01 2.42329929e-02 3.70622426e-01]
 [5.75953066e-01 3.34061943e-02 3.90640736e-01]
 [5.54856896e-01 4.82206270e-02 3.96922469e-01]
 [5.09712994e-01 7.68761262e-02 4.13410872e-01]
 [4.60386068e-01 1.02112278e-01 4.37501580e-01]
 [4.57944989e-01 1.06830738e-01 4.35224265e-01]
 [4.55852568e-01 1.18887857e-01 4.25259501e-01]
 [3.91400069e-01 1.62651882e-01 4.45948035e-01]
 [2.62017608e-01 2.25450665e-01 5.12531698e-01]
 [1.50388703e-01 2.69285768e-01 5.80325544e-01]
 [8.29081610e-02 2.70328522e-01 6.46763325e-01]
 [6.02644272e-02 2.47398183e-01 6.92337394e-01]
 [4.60460968e-02 2.22356111e-01 7.31597722e-01]
 [4.21178751e-02 2.07817346e-01 7.50064731e-01]
 [3.86112332e-02 2.09060550e-01 7.52328157e-01]
 [3.54652815e-02 2.14914471e-01 7.49620318e-01]
 [3.29870619e-02 2.10670948e-01 7.563419

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
# open a single image to visually inspect
#

img_arr = (classes[3] * 255).reshape(256, 256)
print(img_arr.shape)
img = Image.fromarray(img_arr)
img.show()

(256, 256)


In [26]:
# clear the predictions directory and save
# the new predictions
#
clear_old_predictions(predictions_dir)

save_tag = "test_mask"

i = 1
for c in classes:
    img = get_mask_from_binary_array(c)
    img = img.resize((250, 250))
    img.save(os.path.join(predictions_dir, "{}_{}.png".format(save_tag, str(i))), format="PNG")
    i += 1

In [25]:
# sound alarm when model finishes training
#

duration = 5000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)